In [9]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.cifar10 import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot
import numpy as np
import cv2

In [7]:
def msg_to_bin(msg):
    if type(msg)==str:
        return ''.join([format(ord(i),"08b")for i in msg])
    elif type(msg)==bytes or type(msg)==np.ndarray:
        return [format(i,"08b") for i in msg]
    elif type(msg)==int or type(msg)==np.unit8:
        return format(msg,"08b")
    else: 
        raise TypeError("input type not supported")

In [8]:
def hide_message(img,text):
    nbytes=img.shape[0]*img.shape[1]*3//8
    print("maximum bytes ",nbytes)
    if len(text)>nbytes:
        raise ValueError("cross")
    text+="#####"
    dataindex=0
    bin_secret_msg=msg_to_bin(text)
    datalen=len(bin_secret_msg)
    for values in img:
        for pixels in values:
            r,g,b=msg_to_bin(pixels)
            if dataindex<datalen:
                pixels[0]=int(r[:-1]+bin_secret_msg[dataindex],2)
                dataindex+=1
            if dataindex<datalen:
                pixels[1]=int(g[:-1]+bin_secret_msg[dataindex],2)
                dataindex+=1
            if dataindex<datalen:
                pixels[2]=int(b[:-1]+bin_secret_msg[dataindex],2)
                dataindex+=1
            if dataindex>=datalen:
                break
    return img

In [3]:
# define the standalone discriminator model
def define_discriminator(in_shape=(32,32,3)):
	model = Sequential()
	# normal
	model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# classifier
	model.add(Flatten())
	model.add(Dropout(0.4))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

# define the standalone generator model
def define_generator(latent_dim):
	model = Sequential()
	# foundation for 4x4 image
	n_nodes = 256 * 4 * 4
	model.add(Dense(n_nodes, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((4, 4, 256)))
	# upsample to 8x8
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 16x16
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 32x32
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# output layer
	model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
	return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

# load and prepare cifar10 training images
def load_real_samples():
	# load cifar10 dataset
	(trainX, _), (_, _) = load_data()
	# convert from unsigned ints to floats
	X = trainX.astype('float32')
	# scale from [0,255] to [-1,1]
	X = (X - 127.5) / 127.5
	return X

# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	X1=[]
	for i in X:
		i=cv2.resize(i,(i.shape[0],i.shape[1]))
		X1.append(hide_message(i))
	X1=np.array(X1)
	# generate 'real' class labels (1)
	y = ones((n_samples, 1))
	return X1, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = g_model.predict(x_input)
	# create 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y

# create and save a plot of generated images
def save_plot(examples, epoch, n=7):
	# scale from [-1,1] to [0,1]
	examples = (examples + 1) / 2.0
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i])
	# save plot to file
	filename = 'generated_plot_e%03d.png' % (epoch+1)
	pyplot.savefig(filename)
	pyplot.close()

# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
	# prepare real samples
	X_real, y_real = generate_real_samples(dataset, n_samples)
	# evaluate discriminator on real examples
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	# save plot
	save_plot(x_fake, epoch)
	# save the generator model tile file
	filename = 'generator_model_%03d.h5' % (epoch+1)
	g_model.save(filename)

# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=200, n_batch=128):
	bat_per_epo = int(dataset.shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# update discriminator model weights
			d_loss1, _ = d_model.train_on_batch(X_real, y_real)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update discriminator model weights
			d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
			# prepare points in latent space as input for the generator
			X_gan = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch(X_gan, y_gan)
			# summarize loss on this batch
			print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
				(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
		# evaluate the model performance, sometimes
		if (i+1) % 10 == 0:
			summarize_performance(i, g_model, d_model, dataset, latent_dim)

# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

2023-01-07 20:56:42.716752: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-07 20:56:42.717287: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-07 20:56:42.717362: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Lenovoideapad32015ISK6c315432): /proc/driver/nvidia/version does not exist
2023-01-07 20:56:42.719435: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/hrithikpaul/.local/lib/python3.8/site-packages/k

2/2 [==============================] - 1s 234ms/step
>1, 1/390, d1=0.693, d2=0.697 g=0.690
2/2 [==============================] - 0s 221ms/step
>1, 2/390, d1=0.625, d2=0.700 g=0.687
2/2 [==============================] - 0s 225ms/step
>1, 3/390, d1=0.561, d2=0.708 g=0.680
2/2 [==============================] - 0s 218ms/step
>1, 4/390, d1=0.464, d2=0.725 g=0.666
2/2 [==============================] - 0s 224ms/step
>1, 5/390, d1=0.379, d2=0.759 g=0.643
2/2 [==============================] - 0s 243ms/step
>1, 6/390, d1=0.268, d2=0.817 g=0.622
2/2 [==============================] - 0s 233ms/step
>1, 7/390, d1=0.194, d2=0.853 g=0.627
2/2 [==============================] - 0s 227ms/step
>1, 8/390, d1=0.134, d2=0.825 g=0.694
2/2 [==============================] - 0s 229ms/step
>1, 9/390, d1=0.100, d2=0.725 g=0.793
2/2 [==============================] - 0s 216ms/step
>1, 10/390, d1=0.136, d2=0.659 g=0.850
2/2 [==============================] - 0s 216ms/step
>1, 11/390, d1=0.118, d2=0.659 g=0.8

KeyboardInterrupt: 

In [ ]:
def show_data(img):
    bin_data=""
    for values in img:
        for pixels in values:
            r,g,b=msg_to_bin(pixels)
            bin_data+=r[-1]
            bin_data+=g[-1]
            bin_data+=b[-1]
        allbytes=[bin_data[i:i+8]for i in range(0,len(bin_data),8)]
        decodeddata=""
    for bytes in allbytes:
        decodeddata+=chr(int(bytes,2))
        if decodeddata[:-5]=="#####":
            break
    return decodeddata